In [ ]:
%matplotlib inline
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

# MPS Safety Limit Calculations

The LCLS-II MPS system needs to protect components against two categories of damage; instant single shot damage from a single pulse, and ablation via high repetition rate hits. Each **individual** state of each **unique** device along the beamline will have different limitations for thes values based on the material and position along the beamline. 

The purpose of this notebook is to both calculate and document the process in which component information is converted to MPS requirements that are enforced on both the beamline attenuation and the repetition rate of the beam.

## Specified Requirements

The mechanical engineer responsible for the system is also responsible for communicating MPS requirements to the PCDS team. We require two major pieces of information; a table of **maximum single pulse energy** the device can withstand at different photon energies, and the maximum integrated power acceptable for the device. 

The mechanical engineer is welcome to put safety thresholds here in order to reflect the cost of damaging a component. For instance, if a vendor supplied specification claims a device can absorb 10 Watts without issue, but replacing the component would cost days of beamtime. It is at this stage that our table would reflect a large of "safety margin" and allow 8 Watts of power.

In [ ]:
# Load the requirements files via CSV
max_reqs = pd.read_csv(req_file, index_col=0)
max_pulse = pd.read_csv('beam.csv', index_col=0)
state_name = max_reqs.index[0]

In [ ]:
# Create empty versions of the final tables we hope to create
min_attenuation = pd.DataFrame(0., index=max_pulse.index, columns=max_pulse.columns)
max_pulse_rate = pd.DataFrame(0., index=max_pulse.index, columns=max_pulse.columns)

In [ ]:
max_reqs

In [ ]:
fig, ax = plt.subplots()
energy = max_reqs.drop('Power', axis=1)
ax.plot([float(idx) for idx in energy.columns],
        energy.loc[state_name])
labels = ax.set(title='Allowable Single Pulse Energy',
                ylabel='Pulse Energy (mJ)',
                xlabel='Photon Energy (keV)')


## Beam Power Assumptions

Without reliable enough diagnostics, the MPS system has to make assumptions about how powerful the beam can be in certain Accelerator configurations. The table below displays a look-up table filled via theoretical calculation. The two inputs to the table are:

* ** Bunch Charge**: The electron bunch charge travelling through the accelerator (mJ)
* ** Photon Energy**: The energy of an individudal photon produced by the undulators (keV)

In [ ]:
max_pulse

## Calculations

### Required Attenuation
In order to mitigate single shot damage to components, the proper amount of attenuation must be inserted in the beam. This value is determined by a simple ratio of the maximum pulse energy the device can withstand to that predicted at the specific combination of bunch charge and wavelength. This gives us maximum allowable transmission which is simply the inverse of the required attenuation

$$Minimum\,Attenuation = 1 - \frac{Maximum\,Single\,Shot\,Allowed}{Predicted\,Pulse\,Energy}$$

In [ ]:
# Determine the required attenuation at every combination of bunch charge and 
for pulse_energy in max_pulse.columns:
    # Allowed transmission is the maximum allowed pulse energy / predicted amount of pulse energy
    min_attenuation[pulse_energy] = (max_reqs.loc[state_name][pulse_energy]/
                                     (max_pulse[pulse_energy]/1000.)) # Convert to kJ

# No such thing as allowing more than 100% transmission
min_attenuation[min_attenuation > 1.] = 1.
# Modify precision
min_attenuation = min_attenuation.round(2)
# 1 - max_transmission = minimum_attenuation
min_attenuation = 1 - min_attenuation
min_attenuation

### Maximum Repetition Rate

We now know the maximum amount of energy we will be letting through on each shot, but in order to prevent damage from multiple shots we need to limit the repetition rate of the accelerator. The power the component can withstand is provided to us, dividing this by the amount of power we allow per pulse gives us an upper limit that we will allow the repetition rate to be.

$$ Allowable\,Repetition\,Rate = \frac{Maximum Power}{Maximum\,Pulse\,Energy\,After\,Attenuation}$$

In [ ]:
for pulse_energy in max_pulse.columns:
    max_pulse_rate[pulse_energy] = (max_reqs.loc[state_name]['Power']/
                                    (max_pulse[pulse_energy]/1000000.) / 1 - min_attenuation[pulse_energy])
max_pulse_rate = max_pulse_rate.round(0)
max_pulse_rate

## Store Requirements
These requirements are used by the MPS system to protect devices during beamtime. They need to be output to a format readable by a Beckhoff PLC system

In [ ]:
filename_prefix = f'{state_name}'.replace(' ','_')
min_attenuation.to_csv(filename_prefix + '_min_attenuation.csv')
max_pulse_rate.to_csv(filename_prefix + '_max_rep_rate.csv')